In [5]:
import pandas as pd
import numpy as np
import os
import random

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

In [8]:
import duckdb

train_df = duckdb.query('''
    SELECT *
    FROM '../data/raw/train.parquet'
    LIMIT 200000
    '''
).to_df()


In [7]:
duck_train

,gender,age_group,inventory_id,day_of_week,hour,seq,l_feat_1,l_feat_2,l_feat_3,l_feat_4,...,history_b_22,history_b_23,history_b_24,history_b_25,history_b_26,history_b_27,history_b_28,history_b_29,history_b_30,clicked
0,1.0,7.0,36,5,13,"9,18,269,516,57,97,527,74,317,311,269,479,57,7...",1.0,2.0,1.0,23.0,...,0.070092,0.070092,0.011682,0.004673,0.087226,0.049843,0.015576,0.040498,0.051401,0
1,1.0,7.0,2,5,08,"9,144,269,57,516,97,527,74,315,317,311,269,479...",2.0,2.0,3.0,17.0,...,0.072990,0.072990,0.012165,0.004866,0.045416,0.051904,0.016220,0.042172,0.026763,0
2,1.0,7.0,36,5,11,"269,516,57,97,165,527,74,77,317,269,75,450,15,...",1.0,2.0,1.0,7.0,...,0.057177,0.057177,0.009530,0.003812,0.035577,0.081318,0.012706,0.033036,0.062898,0
3,1.0,8.0,37,5,11,"269,57,516,21,214,269,561,214,269,561,247,516,...",2.0,2.0,2.0,7.0,...,0.100449,0.100449,0.016741,0.006697,0.062502,0.071430,0.022322,0.058037,0.073659,0
4,2.0,7.0,37,5,07,"144,269,57,516,35,479,57,516,527,74,77,318,193...",2.0,2.0,3.0,24.0,...,0.064512,0.064512,0.010752,0.004301,0.040141,0.045875,0.014336,0.037274,0.023654,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1.0,8.0,37,4,08,"9,269,57,516,74,527,318,77,452,207,51,452,49,2...",2.0,2.0,3.0,24.0,...,0.054981,0.054981,0.009163,0.003665,0.034210,0.039098,0.012218,0.031767,0.141111,0
199996,2.0,6.0,2,4,08,"9,516,57,27,324,97,516,27,479,27,57,324,516,47...",2.0,2.0,3.0,1.0,...,0.213273,0.213273,0.035546,0.014218,0.132703,0.151661,0.047394,0.123224,0.078200,0
199997,1.0,7.0,36,4,13,"57,516,74,527,317,269,311,479,57,74,315,315,47...",1.0,2.0,1.0,1.0,...,0.113346,0.113346,0.018891,0.007556,0.070526,0.080602,0.025188,0.065489,0.083124,0
199998,2.0,7.0,36,4,12,"9,57,516,527,74,77,318,207,452,452,269,51,508,...",1.0,2.0,1.0,7.0,...,0.043650,0.043650,0.007275,0.002910,0.027160,0.031040,0.009700,0.025220,0.048012,0


### Clicked

In [12]:
train_df['clicked'].value_counts()


clicked
0    196041
1      3959
Name: count, dtype: int64

### seq

## Feat

### l_feat_1

In [ ]:
train_df['l_feat_1'].value_counts()

l_feat_1
2.0    169682
1.0     30318
Name: count, dtype: int64

In [13]:
train_df.groupby('l_feat_1')['clicked'].mean()

l_feat_1
1.0    0.014315
2.0    0.020774
Name: clicked, dtype: float64

### l_feat_2

In [14]:
train_df['l_feat_2'].value_counts()

l_feat_2
2.0    166969
1.0     32547
Name: count, dtype: int64

In [15]:
train_df.groupby('l_feat_2')['clicked'].mean()

l_feat_2
1.0    0.019295
2.0    0.019926
Name: clicked, dtype: float64

# History

### history_b_22

In [ ]:
train_df['history_b_22']

history_b_22
2.647062    191
2.500002    190
3.214287    186
1.698111    183
5.000004    181
           ... 
0.084069      1
0.391869      1
0.162306      1
0.056907      1
0.017469      1
Name: count, Length: 6263, dtype: int64